# Making HTTP Requests

In Python, there are many libraries to make HTTP requests. We will use a 3rd-party library called "requests", which is very easy to use. 

Making a "GET" request is as simple as: 

```python
import requests

res = requests.get(url) # returns a "Response" object
res.content # has the "body" of the response
```

You might need to install the requests library! 

You can do that with the following code in a Jupyter cell: 

```python
! pip install requests
```

Or, if you're using anaconda, optionally you can also do: 

```python
! conda install -c anaconda requests
```

## Parsing JSON data

To parse JSON data in Python, we will use the "json" module: 

```python
import json
```

Read more about the module on the [documentation page](https://docs.python.org/3/library/json.html)!

All we care about for this part is the method "loads", which turns JSON data into a Python object: 

```python
json.loads(my_string_encoded_json)
```

## Pokemon API

There is a simple, open API called "pokeapi" that allows us to make requests and see how to use APIs. Like everything, we first look at the documentation: 

https://pokeapi.co/docs/v2.html

In [4]:
# Let's see how to make a get request to the API: 
import requests
import json
import pandas as pd

In [5]:
res = requests.get('https://pokeapi.co/api/v2/berry')
json.loads(res.content)

{'count': 64,
 'next': 'https://pokeapi.co/api/v2/berry?offset=20&limit=20',
 'previous': None,
 'results': [{'name': 'cheri', 'url': 'https://pokeapi.co/api/v2/berry/1/'},
  {'name': 'chesto', 'url': 'https://pokeapi.co/api/v2/berry/2/'},
  {'name': 'pecha', 'url': 'https://pokeapi.co/api/v2/berry/3/'},
  {'name': 'rawst', 'url': 'https://pokeapi.co/api/v2/berry/4/'},
  {'name': 'aspear', 'url': 'https://pokeapi.co/api/v2/berry/5/'},
  {'name': 'leppa', 'url': 'https://pokeapi.co/api/v2/berry/6/'},
  {'name': 'oran', 'url': 'https://pokeapi.co/api/v2/berry/7/'},
  {'name': 'persim', 'url': 'https://pokeapi.co/api/v2/berry/8/'},
  {'name': 'lum', 'url': 'https://pokeapi.co/api/v2/berry/9/'},
  {'name': 'sitrus', 'url': 'https://pokeapi.co/api/v2/berry/10/'},
  {'name': 'figy', 'url': 'https://pokeapi.co/api/v2/berry/11/'},
  {'name': 'wiki', 'url': 'https://pokeapi.co/api/v2/berry/12/'},
  {'name': 'mago', 'url': 'https://pokeapi.co/api/v2/berry/13/'},
  {'name': 'aguav', 'url': 'https

In [2]:
json.loads(res.content)['next']

'https://pokeapi.co/api/v2/berry?offset=20&limit=20'

In [3]:
#To see the next 20 pokemons
res2 = requests.get('https://pokeapi.co/api/v2/berry?offset=20&limit=20')
con2 = json.loads(res2.content)

In [4]:
con2

{'count': 64,
 'next': 'https://pokeapi.co/api/v2/berry?offset=40&limit=20',
 'previous': 'https://pokeapi.co/api/v2/berry?offset=0&limit=20',
 'results': [{'name': 'pomeg', 'url': 'https://pokeapi.co/api/v2/berry/21/'},
  {'name': 'kelpsy', 'url': 'https://pokeapi.co/api/v2/berry/22/'},
  {'name': 'qualot', 'url': 'https://pokeapi.co/api/v2/berry/23/'},
  {'name': 'hondew', 'url': 'https://pokeapi.co/api/v2/berry/24/'},
  {'name': 'grepa', 'url': 'https://pokeapi.co/api/v2/berry/25/'},
  {'name': 'tamato', 'url': 'https://pokeapi.co/api/v2/berry/26/'},
  {'name': 'cornn', 'url': 'https://pokeapi.co/api/v2/berry/27/'},
  {'name': 'magost', 'url': 'https://pokeapi.co/api/v2/berry/28/'},
  {'name': 'rabuta', 'url': 'https://pokeapi.co/api/v2/berry/29/'},
  {'name': 'nomel', 'url': 'https://pokeapi.co/api/v2/berry/30/'},
  {'name': 'spelon', 'url': 'https://pokeapi.co/api/v2/berry/31/'},
  {'name': 'pamtre', 'url': 'https://pokeapi.co/api/v2/berry/32/'},
  {'name': 'watmel', 'url': 'https

In [5]:
con2['results'][0]

{'name': 'pomeg', 'url': 'https://pokeapi.co/api/v2/berry/21/'}

In [6]:
# Challenge: 
# Create a Dataframe with all the Pokemon names and their URLs. 

def get_pokes(url):
    # Make the HTTP request to the given url. 
    # Parse the response as json
    # return the "next" and the "results" (as a 2-tuple!)
    # make sure to return a "falsey" value (such as None)
    # if there is not a "next!"
    
    res = requests.get(url)
    con = json.loads(res.content)
    
    next_page = con['next']
    
    results = con['results']
    
    
    return next_page, results


def catch_em_all(url):
    pokes = []
    
    # While loop! Like a for-loop, 
    # but goes on for an indetermined amount
    # of time:
    while url:
        url, results = get_pokes(url)
        pokes += results
    return pokes
        

# This data is most naturally represented as a list of dictionaries. 
# How can we create a dataframe from a list of dictionaries? 
# Try to find out on your own, from the internet!

# TODO: turn list_of_pokes into a dataframe.

In [7]:
list_of_pokes = catch_em_all('https://pokeapi.co/api/v2/pokemon')

In [8]:
pokes = pd.DataFrame(columns=['name', 'url'])

In [9]:
for p in list_of_pokes:
    pokes = pokes.append(p, ignore_index =True )

In [10]:
pokes

,name,url
0,bulbasaur,https://pokeapi.co/api/v2/pokemon/1/
1,ivysaur,https://pokeapi.co/api/v2/pokemon/2/
2,venusaur,https://pokeapi.co/api/v2/pokemon/3/
3,charmander,https://pokeapi.co/api/v2/pokemon/4/
4,charmeleon,https://pokeapi.co/api/v2/pokemon/5/
...,...,...
959,araquanid-totem,https://pokeapi.co/api/v2/pokemon/10153/
960,togedemaru-totem,https://pokeapi.co/api/v2/pokemon/10154/
961,necrozma-dusk,https://pokeapi.co/api/v2/pokemon/10155/
962,necrozma-dawn,https://pokeapi.co/api/v2/pokemon/10156/


## Project: Live Exchange Rates

Imagine that you work with financial assets which are denominated in different currencies. You analyze this data regularly, and want to create a "transformation" function that transforms all your assets into EUR prices, based on today's exchange rate. 

Your data with the local-currency-denominated value of each asset lives in a file called "assets.csv" which should be located in the same folder as this notebook. 

Write a "data loading" function that: 

1. Reads the data, given the path to the file. 
2. Returns a dataframe with an additional column that has the assets value in euros, as of today.

Use this free API to get today's exchange rates: https://exchangeratesapi.io/. You will need to read the documentation and try it out to see how it works. 

HINT: Write a separate function to get the current exchange rates! That can be reused!

### Understanding the project

Before creating the function that solves the problem, lets play around with the elements of the problem

In [14]:
df = pd.read_csv('assets.csv')

In [17]:
df

,value,curr
0,48.910052,THB
1,16.505115,THB
2,30.370579,INR
3,14.126967,SEK
4,23.406904,HKD
...,...,...
995,13.593894,HRK
996,41.710860,ZAR
997,12.877760,AUD
998,29.561696,KRW


In [33]:
# Request latest exchange rate data of all currencies agaisnt the euro
# Modify url with ?base=USD to get data for another currency
res = requests.get('https://api.exchangeratesapi.io/latest')

In [42]:
content = json.loads(res.content)

In [43]:
rates = content['rates']

In [46]:
rates

{'CAD': 1.4682,
 'HKD': 8.7298,
 'ISK': 138.1,
 'PHP': 56.286,
 'DKK': 7.4712,
 'HUF': 328.33,
 'CZK': 25.514,
 'AUD': 1.6151,
 'RON': 4.7547,
 'SEK': 10.6993,
 'IDR': 15640.93,
 'INR': 78.816,
 'BRL': 4.4437,
 'RUB': 71.0786,
 'HRK': 7.46,
 'JPY': 120.43,
 'THB': 33.623,
 'CHF': 1.1013,
 'SGD': 1.5129,
 'PLN': 4.2535,
 'BGN': 1.9558,
 'TRY': 6.3761,
 'CNY': 7.844,
 'NOK': 10.1638,
 'NZD': 1.7326,
 'ZAR': 16.828,
 'USD': 1.1139,
 'MXN': 21.3164,
 'ILS': 3.9272,
 'GBP': 0.86008,
 'KRW': 1300.09,
 'MYR': 4.64}

In [12]:
def to_euro(rates, curr, amount):
    """Function that searchs for the specified currency in an exchange rate dictionary 
    and converts it into euros.
    
    Args:
        rates (dict): A dictionary containing the exchange rates of several currencies to euro
        curr (str): Abbreviation of the currency to be converted.
        amount (float): Total amount of the currency to be converted.

    Returns:
        A float with the converted amounted into euros. None if the currency was not found.
    """
    
    try:
        exchanged = amount/rates[curr]
    except KeyError:
        print('Currency not found, returned a missing value')
        exchanged = None

    return(exchanged)

In [16]:
def load_data(path):
    """Function that loads a CSV file from the specified path as a Pandas DataFrame
    and adds to it a new column (value_euros) with the value in euros of the assets
    found in the CSV. Information on exchange rates is taken from the following API:
    https://api.exchangeratesapi.io
    
    Args:
        path (str): A path that specifies the location of a CSV file.

    Returns:
        A Pandas DataFrame object with a new column (value_euros).
    """
    
    #Loading the CSV file from the path as a Pandas DataFrame
    df = pd.read_csv(path)
    
    #Setting up connection with the API and extracting the exchange rates data
    res = requests.get('https://api.exchangeratesapi.io/latest')
    content = json.loads(res.content)
    rates = content['rates']
    
    #Iterating over the elements of the DataFrame
    exchanged = []
    for i in range(len(df)):
        exchanged.append(to_euro(rates, df.curr[i], df.value[i]))
        
    df['value_euros'] = exchanged
    return(df)
    

In [17]:
df_e = load_data('assets.csv')

In [18]:
df_e

,value,curr,value_euros
0,48.910052,THB,1.454661
1,16.505115,THB,0.490888
2,30.370579,INR,0.385335
3,14.126967,SEK,1.320364
4,23.406904,HKD,2.681265
...,...,...,...
995,13.593894,HRK,1.822238
996,41.710860,ZAR,2.478658
997,12.877760,AUD,7.973352
998,29.561696,KRW,0.022738
